In [2]:
import sympy
from sympy.plotting import plot
import sympy as sp
import penbegone as pbg
from penbegone import common as bgcom
from penbegone import plotting as bgplot
from penbegone.common import printeq

%matplotlib inline
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

# ArduPilot TECS mathematical model

This is just a simplification.

Nomenclature:
- $h$: Height, $m$
- $\dot{h}$: Climb rate, $m/s$
- $V$: Airspeed, $m/s$
- $V_d$: Demanded airspeed, $m/s$
- $\dot{V}$: Airspeed derivative, $m/s^2$

Parameters:
- $V_{max}$: Maximum airspeed, $m/s$
- $V_{min}$: Minimum airspeed, $m/s$
- $\dot{h}_{c,max}$: Maximum climb rate, $m/s$
- $\dot{h}_{s,max}$: Maximum sink rate, $m/s$
- $\dot{h}_{s,min}$: Minimum sink rate, $m/s$

Constants:
- $g$: Acceleration of gravity, $m/s^2$

In [3]:
t = sp.symbols("t", real=True)
h, h_dem, v, v_dem = bgcom.functions("h h_d v v_dem", t)
h_dot, v_c, v_dot = sp.symbols("hdot v_cruise vdot")
v_max, v_min, climb_max, sink_max, sink_min = sp.symbols("V_max V_min hdot_cmax hdot_smax hdot_smin")
thr_max, thr_min, thr_trim = sp.symbols("t_{max} t_{min} t_{trim}")
w, k_tc, k_pd, k_i, k_td = sp.symbols("w k_tc k_pd k_i k_td", real=True)
g = sp.symbols("g", real=True)

h_dot_ = sp.diff(h, t) 
v_dot_ = sp.diff(v, t)

In [4]:
STEdot_max, STEdot_min, STEdot_neg_max = sp.symbols("Edot^*_max Edot^*_min Edot^*_nmax")

STEdot_max_ = g*climb_max
STEdot_min_ = -g*sink_min
STEdot_neg_max = -g*sink_max

In [5]:
## _update_speed_demand

# Perhaps the increase of _TAS_dem when at max sink plays a role?

vel_rate_max_ = 0.5*STEdot_max_/v
vel_rate_neg_cruise = 0.9*STEdot_min_/v_c
vel_rate_neg_max = 0.9*STEdot_min_/v_max

# I can add the LPF effect of TIME_CONST here, instead of pure differentiation.
vdot_dem_ = sp.diff(v_dem, t)

In [6]:
## _update_height_demand

# Can add the LPF effect of HDEM_TCONST here, over h_dem.
h_dot_dem_ = sp.diff(h_dem)


In [7]:
## _update_energies

SPE_dem_ = h_dem*g
SKE_dem_ = 0.5*v_dem*v_dem
SKE_est_ = 0.5*v*v
SKE_dot_dem_= v*vdot_dem_ # In the source vdot_dem_ is high-passed. How does this translate to the actual system?
SKE_dot_ = v*v_dot # Same comment as above.
SPE_est_ = h*g
SPE_dot_ = h_dot*g

In [8]:
## _update_pitch

w_k_ = w
w_p_ = 2-w

SEB_dem_ = SPE_dem_*w_p_ - SKE_dem_*w_k_
SEB_est_ = SPE_est_*w_p_ - SKE_est_*w_k_
SEB_err_ = SEB_dem_ - SEB_est_
SEB_dot_est_ = SPE_dot_*w_p_ - SKE_dot_*w_k_

SEB_dot_dem_ = SEB_err_/k_tc + h_dot_dem_*g*w_p_
SEB_dot_err_ = SEB_dot_dem_ - SEB_dot_est_
SEB_dot_dem_tot_ = SEB_dot_dem_ + SEB_dot_err_*k_pd

SEB_dot_int_ = k_i*sp.integrate(SEB_dot_err_, t)
KE_int_ = sp.integrate((SKE_est_ - SKE_dem_)*w_k_/k_tc,t)

pitch_dem_ = (SEB_dot_dem_tot_ + SEB_dot_int_ + KE_int_) / (v * g)

In [9]:
## _update_throttle_with_airspeed

STE_error_ = SKE_dem_ - SKE_est_ + SPE_dem_ - SPE_est_
SPE_dot_dem_ = (SPE_dem_ - SPE_est_)/k_tc
STE_dot_dem_ = SPE_dot_dem_ + SKE_dot_dem_
STE_dot_err_ = STE_dot_dem_ - SKE_dot_ - SPE_dot_

K_thr2STE_ = (STEdot_max_ - STEdot_min_)/(thr_max - thr_min)
K_STE2Thr_ = 1 / (K_thr2STE_ * k_tc)
integTHR_state_ = sp.integrate(STE_error_*k_i*K_STE2Thr_,t)
ff_throttle = STE_dot_dem_/K_thr2STE_ + thr_trim

throttle_dem_ = K_STE2Thr_*(STE_error_ + STE_dot_err_*k_td) + ff_throttle + integTHR_state_

In [32]:
throttle_dem_.simplify()

(g*k_tc**2*t_{trim}*(hdot_cmax + hdot_smin) - 1.0*k_i*k_tc*(t_{max} - t_{min})*(1.0*g*Integral(h(t), t) - 1.0*g*Integral(h_d(t), t) + 0.5*Integral(v(t)**2, t) - 0.5*Integral(v_dem(t)**2, t)) + k_tc*(t_{max} - t_{min})*(-g*h(t) + g*h_d(t) + k_tc*v(t)*Derivative(v_dem(t), t)) - (t_{max} - t_{min})*(k_tc*(g*h(t) - g*h_d(t) + 0.5*v(t)**2 - 0.5*v_dem(t)**2) + k_td*(g*h(t) - g*h_d(t) + k_tc*(g*hdot + vdot*v(t) - v(t)*Derivative(v_dem(t), t)))))/(g*k_tc**2*(hdot_cmax + hdot_smin))

## Kinematic model

In [11]:
pitch, throttle, m = sp.symbols("theta delta_t m")
F_t, c_drag, S, rho = sp.symbols("F_t C_drag S rho")
drag_ = 0.5*S*rho*v*v*c_drag
f_v_dot_ = 1/m*(sp.sin(pitch)*g + F_t*throttle - drag_)
f_h_dot_ = v*sp.sin(pitch)  # Perhaps add effect of increasing lift as airspeed increases?

## System definition

In [12]:
fcl_1 = f_v_dot_.subs({pitch: pitch_dem_, throttle: throttle_dem_})
fcl_2 = f_h_dot_.subs({pitch: pitch_dem_})

In [13]:
fcl_1.simplify()

(F_t*(g*k_tc**2*t_{trim}*(hdot_cmax + hdot_smin) - 1.0*k_i*k_tc*(t_{max} - t_{min})*(1.0*g*Integral(h(t), t) - 1.0*g*Integral(h_d(t), t) + 0.5*Integral(v(t)**2, t) - 0.5*Integral(v_dem(t)**2, t)) + k_tc*(t_{max} - t_{min})*(-g*h(t) + g*h_d(t) + k_tc*v(t)*Derivative(v_dem(t), t)) - (t_{max} - t_{min})*(k_tc*(g*h(t) - g*h_d(t) + 0.5*v(t)**2 - 0.5*v_dem(t)**2) + k_td*(g*h(t) - g*h_d(t) + k_tc*(g*hdot + vdot*v(t) - v(t)*Derivative(v_dem(t), t))))) - g*k_tc**2*(hdot_cmax + hdot_smin)*(0.5*C_drag*S*rho*v(t)**2 + g*sin((g*k_tc*(w - 2)*Derivative(h_d(t), t) - g*(w - 2)*h(t) + g*(w - 2)*h_d(t) + 2.0*k_i*(-0.5*g*hdot*k_tc*t*w + 1.0*g*hdot*k_tc*t + 0.5*g*k_tc*w*h_d(t) - 1.0*g*k_tc*h_d(t) - 0.5*g*w*Integral(h(t), t) + 0.5*g*w*Integral(h_d(t), t) + 1.0*g*Integral(h(t), t) - 1.0*g*Integral(h_d(t), t) - 0.5*k_tc*vdot*w*Integral(v(t), t) - 0.25*w*Integral(v(t)**2, t) + 0.25*w*Integral(v_dem(t)**2, t)) - k_pd*(g*(w - 2)*h(t) - g*(w - 2)*h_d(t) + k_tc*(g*hdot*(w - 2) - g*(w - 2)*Derivative(h_d(t), t) + 

In [14]:
fcl_2.simplify()

v(t)*sin((-g*k_tc*(w - 2)*Derivative(h_d(t), t) + g*(w - 2)*h(t) - g*(w - 2)*h_d(t) + 2.0*k_i*(0.5*g*hdot*k_tc*t*w - 1.0*g*hdot*k_tc*t - 0.5*g*k_tc*w*h_d(t) + 1.0*g*k_tc*h_d(t) + 0.5*g*w*Integral(h(t), t) - 0.5*g*w*Integral(h_d(t), t) - 1.0*g*Integral(h(t), t) + 1.0*g*Integral(h_d(t), t) + 0.5*k_tc*vdot*w*Integral(v(t), t) + 0.25*w*Integral(v(t)**2, t) - 0.25*w*Integral(v_dem(t)**2, t)) + k_pd*(g*(w - 2)*h(t) - g*(w - 2)*h_d(t) + k_tc*(g*hdot*(w - 2) - g*(w - 2)*Derivative(h_d(t), t) + vdot*w*v(t)) + 0.5*w*v(t)**2 - 0.5*w*v_dem(t)**2) + 0.5*w*(Integral(v(t)**2, t) - Integral(v_dem(t)**2, t)) + 0.5*w*v(t)**2 - 0.5*w*v_dem(t)**2)/(g*k_tc*v(t)))

### Transformation onto trim

In [15]:
v_star, v_dem_star, h_star = sp.symbols("v^* v^*_dem h^*")
fclt_1 = fcl_1.subs({v: v_star+v_c, v_dem: v_dem_star+v_c})
fclt_2 = fcl_2.subs({v: v_star+v_c, v_dem: v_dem_star+v_c})

In [16]:
fclt_1.simplify()

(F_t*(g*k_tc**2*t_{trim}*(hdot_cmax + hdot_smin) - 1.0*k_i*k_tc*(t_{max} - t_{min})*(1.0*g*Integral(h(t), t) - 1.0*g*Integral(h_d(t), t) + 0.5*t*(v^* + v_cruise)**2 - 0.5*t*(v^*_dem + v_cruise)**2) + k_tc*(t_{max} - t_{min})*(-g*h(t) + g*h_d(t)) - (t_{max} - t_{min})*(k_tc*(g*h(t) - g*h_d(t) + 0.5*(v^* + v_cruise)**2 - 0.5*(v^*_dem + v_cruise)**2) + k_td*(g*h(t) - g*h_d(t) + k_tc*(g*hdot + vdot*(v^* + v_cruise))))) - g*k_tc**2*(hdot_cmax + hdot_smin)*(0.5*C_drag*S*rho*(v^* + v_cruise)**2 + g*sin((g*k_tc*(w - 2)*Derivative(h_d(t), t) - g*(w - 2)*h(t) + g*(w - 2)*h_d(t) + 2.0*k_i*(-0.5*g*hdot*k_tc*t*w + 1.0*g*hdot*k_tc*t + 0.5*g*k_tc*w*h_d(t) - 1.0*g*k_tc*h_d(t) - 0.5*g*w*Integral(h(t), t) + 0.5*g*w*Integral(h_d(t), t) + 1.0*g*Integral(h(t), t) - 1.0*g*Integral(h_d(t), t) - 0.5*k_tc*t*vdot*w*(v^* + v_cruise) - 0.25*t*w*(v^* + v_cruise)**2 + 0.25*t*w*(v^*_dem + v_cruise)**2) - k_pd*(g*(w - 2)*h(t) - g*(w - 2)*h_d(t) + k_tc*(g*hdot*(w - 2) - g*(w - 2)*Derivative(h_d(t), t) + vdot*w*(v^* + 

In [17]:
fclt_2.simplify()

(-v^* - v_cruise)*sin((g*k_tc*(w - 2)*Derivative(h_d(t), t) - g*(w - 2)*h(t) + g*(w - 2)*h_d(t) + 2.0*k_i*(-0.5*g*hdot*k_tc*t*w + 1.0*g*hdot*k_tc*t + 0.5*g*k_tc*w*h_d(t) - 1.0*g*k_tc*h_d(t) - 0.5*g*w*Integral(h(t), t) + 0.5*g*w*Integral(h_d(t), t) + 1.0*g*Integral(h(t), t) - 1.0*g*Integral(h_d(t), t) - 0.5*k_tc*t*vdot*w*(v^* + v_cruise) - 0.25*t*w*(v^* + v_cruise)**2 + 0.25*t*w*(v^*_dem + v_cruise)**2) - k_pd*(g*(w - 2)*h(t) - g*(w - 2)*h_d(t) + k_tc*(g*hdot*(w - 2) - g*(w - 2)*Derivative(h_d(t), t) + vdot*w*(v^* + v_cruise)) + 0.5*w*(v^* + v_cruise)**2 - 0.5*w*(v^*_dem + v_cruise)**2) - 0.5*t*w*((v^* + v_cruise)**2 - (v^*_dem + v_cruise)**2) - 0.5*w*(v^* + v_cruise)**2 + 0.5*w*(v^*_dem + v_cruise)**2)/(g*k_tc*(v^* + v_cruise)))

### Is origin a solution?

In [22]:
fclt_1.subs({v_star: 0, v_dot: 0, h: 0, h_dot: 0, v_dem_star: 0, h_dem: 0}).simplify().factor()

-1.0*(0.5*C_drag*S*rho*v_cruise**2 - 1.0*F_t*t_{trim})/m

Basically yes, when thrust is equal to drag.

In [29]:
fclt_2.subs({v_star: 0, v_dot: 0, h: 0, v_dem_star: 0, h_dem: 0, h_dot: 0, h_dot_dem_: 0, v_dot_: 0}).simplify()

0

Also yes.